In [22]:
import tensorflow as tf
tf.enable_eager_execution
from tensorflow import keras
from tensorflow.keras import layers,Sequential
import numpy as np

###creation of layers and model
##batch_input_shape=(numberofSequence,timesteps,data_dim)
l1 = layers.LSTM(units=20,return_sequences=True,batch_input_shape=(None,3,5))
l2 = layers.LSTM(units=12,return_sequences=True)
l3 = layers.Dense(2,activation='softmax')
l4 = layers.Dense(1)

In [23]:
### helper functions
def cal_discounted_rew(rew, disc_factor = 0.9):
    for i in range(rew.shape[0]-2,-1,-1):
        rew[i] += disc_factor *rew[i+1]
    return rew

def cal_advantage(reward_array, state_array,critic,disc_factor=0.9):
    ###parametrize it for generalized advantage estimation
    values_of_states = critic.predict(state_array)
    print("ok1")
    ###reshape this array values_of_states
    values_of_states = values_of_states.reshape((values_of_states.shape[1],))
    reward_array = cal_discounted_rew(reward_array,disc_factor)    
    print("ok2")
    advantage_array = np.zeros(reward_array.shape)
    advantage_array = reward_array - values_of_states
    print("ok3")
    return advantage_array

In [24]:
Actor= Sequential([l1,l2,l3])
critic = Sequential([l1,l2,l4])
critic.compile(loss='mse',optimizer='sgd')
critic.summary()
Actor.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 3, 20)             2080      
_________________________________________________________________
lstm_5 (LSTM)                (None, 3, 12)             1584      
_________________________________________________________________
dense_5 (Dense)              (None, 3, 1)              13        
Total params: 3,677
Trainable params: 3,677
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 3, 20)             2080      
_________________________________________________________________
lstm_5 (LSTM)                (None, 3, 12)             1584      
_________________________________________________________________
dense_4 

In [25]:
seq_len = 3
state_size=5
shape_states=(1,seq_len,state_size)
shape_rew = (1,seq_len,1)
in_data =np.random.normal(1,2,shape_states)
y = np.random.normal(1,0,shape_rew)

y = cal_discounted_rew(y.reshape((seq_len,)))
y = y.reshape(shape_rew)

In [26]:
critic.fit(x=in_data,y=y)

Epoch 1/1
1/1 [==============================] - 1s 1s/step - loss: 3.9364


In [49]:
def mult_loss_by_advantage(y_true,y_pred,advantage):
    ###change size of advantage compatible to y_pred
    #then multiply
    seq_len=3
    action_size = 2
    advantage = advantage.reshape((seq_len,1))
    temp = np.ones((1,action_size))
    advantage = np.matmul(advantage,temp)
    print("multiplication done")
    advantage = advantage.reshape((1,seq_len,action_size))
    loss1 = (tf.log(y_pred)) * (advantage)
    print(loss1)
    return np.sum(loss1)

def actor_loss(advantage):
    #y_act is a one hot vector, actual action that was taken
    #y_pred is softmax output from actor model
    def my_fun(y_true,y_pred):
        return mult_loss_by_advantage(y_true,y_pred,advantage)
    return my_fun

In [50]:
print(critic.predict(in_data))
advantage = cal_advantage(y.reshape((seq_len,)),in_data,critic)
loss_fn = actor_loss(advantage)
Actor.compile(loss =loss_fn,optimizer='sgd' )

[[[0.04892085]
  [0.06274211]
  [0.07400935]]]
ok1
ok2
ok3
multiplication done
Tensor("loss_13/dense_4_loss/mul:0", shape=(?, 3, 2), dtype=float32)


In [51]:
y_true = np.asarray([1,0,0,1,0,1])
y_true = y_true.reshape((1,3,2))
history = Actor.fit(in_data,y_true,epochs=4)

Epoch 1/4
1/1 [==============================] - 2s 2s/step - loss: -10.7209
Epoch 2/4
1/1 [==============================] - 0s 0us/step - loss: -10.7220
Epoch 3/4
1/1 [==============================] - 0s 19ms/step - loss: -10.7232
Epoch 4/4
1/1 [==============================] - 0s 5ms/step - loss: -10.7247


In [52]:
history.history.values()

dict_values([[-10.720915794372559, -10.721985816955566, -10.72324275970459, -10.72472095489502]])